In [ ]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.models.catboost_multiclass import CatBoostMulticlass
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase
from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_models = load_yaml_config("../configs/models.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, _ = PrepareDatasetUseCase(
    repo, schema, cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]


EXPERIMENT_NAME = cfg_mlflow["mlflow"]["experiment_name"]
RUN_NAME = generate_run_name(prefix="catboost")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)


def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def catboost_pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model = CatBoostMulticlass(
            **cfg_models["catboost"],
    ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=catboost_pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "catboost_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

result = use_case.execute(
    X_df=X,
    y=y,
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print("Fold MAP@3:", result.fold_scores)
print("Mean MAP@3:", result.mean_score)

2026-01-09 16:31:54.954 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:36 - Tracking URI : sqlite://///Users/surelmanda/Downloads/ml-projects/mlflow_central/db/mlruns.db
2026-01-09 16:31:54.966 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:37 - Artifact URI : /Users/surelmanda/Downloads/ml-projects/mlflow_central/mlflow_artifacts
2026/01/09 16:31:56 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/09 16:31:56 INFO mlflow.store.db.utils: Updating database tables
2026/01/09 16:31:56 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/09 16:31:56 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/09 16:31:56 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/09 16:31:56 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026-01-09 16:31:56.148 | INFO     | fertilizer_recommender.infrastructure.tracking.mlflow_setup:configur

0:	learn: 1.9456421	total: 575ms	remaining: 5m 44s
100:	learn: 1.9325330	total: 54.1s	remaining: 4m 27s
200:	learn: 1.9235761	total: 1m 42s	remaining: 3m 23s
300:	learn: 1.9154718	total: 2m 19s	remaining: 2m 19s
400:	learn: 1.9079308	total: 3m 10s	remaining: 1m 34s
500:	learn: 1.9008041	total: 3m 53s	remaining: 46.1s


2026-01-09 16:36:34.514 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:95 - [Fold 1] Prédiction et calcul du top-3


599:	learn: 1.8940863	total: 4m 30s	remaining: 0us


2026-01-09 16:36:37.347 | SUCCESS  | fertilizer_recommender.application.use_cases.train_with_cv:execute:106 - [Fold 1] Score MAP@3 = 0.3137
2026-01-09 16:36:37.349 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_fold1': 0.31374777777783786}
2026-01-09 16:36:37.349 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_fold1': 0.31374777777783786}
2026-01-09 16:36:37.647 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:86 - [Fold 2] Démarrage (train=600000 obs, val=150000 obs)
2026-01-09 16:36:37.650 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:91 - [Fold 2] Entraînement du modèle


...

0:	learn: 1.9456366	total: 673ms	remaining: 6m 43s
100:	learn: 1.9324193	total: 34.4s	remaining: 2m 49s
200:	learn: 1.9234422	total: 1m 7s	remaining: 2m 13s
300:	learn: 1.9154473	total: 1m 40s	remaining: 1m 39s
400:	learn: 1.9079277	total: 2m 22s	remaining: 1m 10s
500:	learn: 1.9008030	total: 2m 59s	remaining: 35.5s
599:	learn: 1.8942240	total: 3m 48s	remaining: 0us


2026-01-09 16:49:57.525 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:95 - [Fold 4] Prédiction et calcul du top-3
2026-01-09 16:49:59.981 | SUCCESS  | fertilizer_recommender.application.use_cases.train_with_cv:execute:106 - [Fold 4] Score MAP@3 = 0.3148
2026-01-09 16:49:59.982 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_fold4': 0.31481444444450796}
2026-01-09 16:49:59.982 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_fold4': 0.31481444444450796}
2026-01-09 16:50:00.258 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:86 - [Fold 5] Démarrage (train=600000 obs, val=150000 obs)
2026-01-09 16:50:00.260 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:91 - [Fold 5] Entraînement du modèle


0:	learn: 1.9456361	total: 485ms	remaining: 4m 50s
100:	learn: 1.9323676	total: 1m 1s	remaining: 5m 2s
200:	learn: 1.9235457	total: 1m 39s	remaining: 3m 17s
300:	learn: 1.9155399	total: 2m 12s	remaining: 2m 11s
400:	learn: 1.9080300	total: 2m 44s	remaining: 1m 21s
500:	learn: 1.9008205	total: 3m 17s	remaining: 38.9s
599:	learn: 1.8940803	total: 4m 3s	remaining: 0us


2026-01-09 16:54:09.643 | INFO     | fertilizer_recommender.application.use_cases.train_with_cv:execute:95 - [Fold 5] Prédiction et calcul du top-3
2026-01-09 16:54:13.680 | SUCCESS  | fertilizer_recommender.application.use_cases.train_with_cv:execute:106 - [Fold 5] Score MAP@3 = 0.3135
2026-01-09 16:54:13.681 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_fold5': 0.3134988888889468}
2026-01-09 16:54:13.681 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_fold5': 0.3134988888889468}
2026-01-09 16:54:13.695 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:log_evaluation:46 - Métriques d'évaluation : {'map_3_mean': 0.31401266666672706}
2026-01-09 16:54:13.696 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:log_metrics:86 - Métriques enregistrées : {'map_3_mean': 0.3140126

Fold MAP@3: [0.31374777777783786, 0.3138544444445061, 0.3141477777778367, 0.31481444444450796, 0.3134988888889468]
Mean MAP@3: 0.31401266666672706


In [3]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.models.lightgbm_multiclass import LightGBMMulticlass
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase
from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_models = load_yaml_config("../configs/models.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, _ = PrepareDatasetUseCase(
    repo, schema, cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]


EXPERIMENT_NAME = cfg_mlflow["mlflow"]["experiment_name"]
RUN_NAME = generate_run_name(prefix="Lightgbm")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)


def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def lightgbm_pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model=LightGBMMulticlass(
            #num_class=len(y.unique()),
            **cfg_models["lightgbm"],
        ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=lightgbm_pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "lightgbm_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

result = use_case.execute(
    X_df=X,
    y=y,
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print("Fold MAP@3:", result.fold_scores)
print("Mean MAP@3:", result.mean_score)

2026-01-09 16:59:04.199 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:36 - Tracking URI : sqlite://///Users/surelmanda/Downloads/ml-projects/mlflow_central/db/mlruns.db
2026-01-09 16:59:04.206 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:37 - Artifact URI : /Users/surelmanda/Downloads/ml-projects/mlflow_central/mlflow_artifacts
2026-01-09 16:59:04.280 | INFO     | fertilizer_recommender.infrastructure.tracking.mlflow_setup:configure:53 - MLflow configuré avec succès.
2026-01-09 16:59:04.282 | INFO     | fertilizer_recommender.infrastructure.tracking.mlflow_tracker:__init__:24 - MLflowExperimentTracker prêt.
2026-01-09 16:59:04.285 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:__init__:21 - ExperimentTrackingService initialisé.
2026-01-09 16:59:04.309 | INFO     | fertilizer_recommender.domain.services.experiment_tracking_service:start_experiment:28 - Initialisation de l'expérience

KeyboardInterrupt: 

In [ ]:
from pathlib import Path

from fertilizer_recommender.infrastructure.utils.config_loader import load_yaml_config
from fertilizer_recommender.infrastructure.repositories.dataset_repository_impl import CsvDatasetRepository
from fertilizer_recommender.domain.entities.fertilizer_features import FertilizerFeaturesSchema
from fertilizer_recommender.application.use_cases.prepare_dataset import PrepareDatasetUseCase

from fertilizer_recommender.infrastructure.ml.models.xgboost_multiclass import XGBoostMulticlass
from fertilizer_recommender.infrastructure.tracking.run_name_generator import (
    generate_run_name,
)
from fertilizer_recommender.infrastructure.tracking.mlflow_tracker import MLflowExperimentTracker
from fertilizer_recommender.domain.services.experiment_tracking_service import (
    ExperimentTrackingService
)
from fertilizer_recommender.infrastructure.ml.cv.splitter import make_stratified_kfold
from fertilizer_recommender.application.use_cases.train_with_cv import TrainWithCVUseCase
from fertilizer_recommender.infrastructure.ml.preprocessors.sklearn_transformer import SklearnFeatureTransformer
from fertilizer_recommender.infrastructure.ml.pipelines.training_pipeline import TrainingPipeline


cfg_train = load_yaml_config("../configs/training.yaml")
cfg_models = load_yaml_config("../configs/models.yaml")
cfg_mlflow = load_yaml_config("../configs/mlflow.yaml")

schema = FertilizerFeaturesSchema(
    numeric_features=["Temparature","Humidity","Moisture","Nitrogen","Potassium","Phosphorous"],
    categorical_features=["Soil Type","Crop Type"],
)

repo = CsvDatasetRepository(
    data_dir=Path("../data/raw"),
    train_file=cfg_train["data"]["train_file"],
    test_file=cfg_train["data"]["test_file"],
)

train_df, _ = PrepareDatasetUseCase(
    repo, schema, cfg_train["data"]["target_col"]
).execute()

X = train_df[schema.all_features]
y = train_df[cfg_train["data"]["target_col"]]


EXPERIMENT_NAME = cfg_mlflow["mlflow"]["experiment_name"]
RUN_NAME = generate_run_name(prefix="xgboost")

# 🔑 Wiring propre
tracker = MLflowExperimentTracker()
experiment_service = ExperimentTrackingService(tracker)


def splitter_factory():
    return make_stratified_kfold(
        n_splits=cfg_train["training"]["n_splits"],
        seed=cfg_train["project"]["seed"],
    )

def xgboost_pipeline_factory():
    return TrainingPipeline(
        transformer=SklearnFeatureTransformer(
            schema.numeric_features,
            schema.categorical_features,
        ),
        model=XGBoostMulticlass(
            #num_class=len(y.unique()),
            **cfg_models["xgboost"],
        ),
    )

use_case = TrainWithCVUseCase(
    experiment_service=experiment_service,
    splitter_factory=splitter_factory,
    pipeline_factory=xgboost_pipeline_factory,
    top_k=cfg_train["training"]["top_k"],
)

params = {
    "model": "xgboost_multinomial",
    "n_splits": cfg_train["training"]["n_splits"],
    "top_k": cfg_train["training"]["top_k"],
    "seed": cfg_train["project"]["seed"],
}

def encodelabel(y):
    #import numpy as np
    from typing import Dict, Any

    #print(np.unique(y))
    map_encod : Dict[Any, int] = {
        '10-26-26': 0,
        '14-35-14': 1,
        '17-17-17': 2,
        '20-20': 3,
        '28-28': 4,
        'DAP': 5,
        'Urea': 6,
    }
    return y.map(map_encod)

def process_input_data(X):
    X_transf = X.copy(deep=True)
    
    for col in X_transf.select_dtypes(exclude="number").columns:
        X_transf[col] = X_transf[col].astype("category")
    
    return X_transf


result = use_case.execute(
    X_df=process_input_data(X),
    y=encodelabel(y),
    experiment_name=EXPERIMENT_NAME,
    run_name=RUN_NAME,
    params=params,
)

print("Fold MAP@3:", result.fold_scores)
print("Mean MAP@3:", result.mean_score)

2026-01-09 14:08:55.192 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:36 - Tracking URI : sqlite://///Users/surelmanda/Downloads/ml-projects/mlflow_central/db/mlruns.db
2026-01-09 14:08:55.198 | DEBUG    | fertilizer_recommender.infrastructure.tracking.mlflow_setup:__init__:37 - Artifact URI : /Users/surelmanda/Downloads/ml-projects/mlflow_central/mlflow_artifacts
2026/01/09 14:08:55 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/09 14:08:55 INFO mlflow.store.db.utils: Updating database tables
2026/01/09 14:08:55 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/09 14:08:55 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/09 14:08:55 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/09 14:08:55 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026-01-09 14:08:55.929 | INFO     | fertilizer_recommender.infrastructure.tracking.mlflow_setup:configur

Fold MAP@3: [0.33875000000008243, 0.33972777777786334, 0.3397466666667503, 0.3393622222223047, 0.34028444444452743]
Mean MAP@3: 0.33957422222230566


In [ ]:
from fertilizer_recommender.application.use_cases.evaluate_model import EvaluateModelUseCase

#evaluator = EvaluateModelUseCase(pipeline, top_k=3)
#metrics = evaluator.execute(X_train, y_train.tolist())
#metrics

In [ ]:
#storage = LocalStorage(Path("../artifacts/reports"))
#storage.save_text("hello", "test.txt")
#storage.load_text("test.txt")